# API交互
- 如果你需要的数据或操作在API之后，就需要LLM能够和API进行交互
- 到这个环节，就与Agents和Plugins息息相关了
- Demo很交单，但是功能复杂

In [1]:


# here put the import lib
from typing import Any, List, Mapping, Optional, Dict
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from zhipuai import ZhipuAI

import os

# 继承自 langchain.llms.base.LLM
class ZhipuAILLM(LLM):
    # 默认选用 glm-3-turbo
    model: str = "glm-3-turbo"
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = "acf4f9247da5e232fbe056b14b35fd9b.uWW0WvWqwWUYjhzQ"
    
    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        client = ZhipuAI(
            api_key = self.api_key
        )

        def gen_glm_params(prompt):
            '''
            构造 GLM 模型请求参数 messages

            请求参数：
                prompt: 对应的用户提示词
            '''
            messages = [{"role": "user", "content": prompt}]
            return messages
        
        messages = gen_glm_params(prompt)
        response = client.chat.completions.create(
            model = self.model,
            messages = messages,
            temperature = self.temperature
        )

        if len(response.choices) > 0:
            return response.choices[0].message.content
        return "generate answer error"


    # 首先定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        """获取调用API的默认参数。"""
        normal_params = {
            "temperature": self.temperature,
            }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _llm_type(self) -> str:
        return "Zhipu"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model": self.model}, **self._default_params}

In [4]:
llm = ZhipuAILLM()

In [8]:
import os
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs

chain = APIChain.from_llm_and_api_docs(llm=llm, api_docs=open_meteo_docs.OPEN_METEO_DOCS, verbose=True, limit_to_domains=None)
chain.run('What is the weather like right now in Munich, Germany in degrees Fahrenheit?')

D:\Python\envs\py310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new APIChain chain...
To answer the user's question about the current weather in Munich, Germany in degrees Fahrenheit, we would need to include the `current_weather` parameter and set it to `true`, the `temperature_unit` parameter and set it to `fahrenheit`, and the `latitude` and `longitude` parameters for the Munich location. The API URL would look like this:

```
https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.582
```

However, since the user specifically asked for the current weather conditions, we don't need to include hourly or daily parameters, as these would provide additional weather data beyond the current conditions. The final URL to query the API for the current weather in Munich in Fahrenheit would be:

```
https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.582&current_weather=true&temperature_unit=fahrenheit
```


InvalidSchema: No connection adapters were found for "To answer the user's question about the current weather in Munich, Germany in degrees Fahrenheit, we would need to include the `current_weather` parameter and set it to `true`, the `temperature_unit` parameter and set it to `fahrenheit`, and the `latitude` and `longitude` parameters for the Munich location. The API URL would look like this:\n\n```\nhttps://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.582\n```\n\nHowever, since the user specifically asked for the current weather conditions, we don't need to include hourly or daily parameters, as these would provide additional weather data beyond the current conditions. The final URL to query the API for the current weather in Munich in Fahrenheit would be:\n\n```\nhttps://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.582&current_weather=true&temperature_unit=fahrenheit\n```"